In [1]:
import sys, os
sys.path.append('..')

# Verify the path is correct
print(f"Current directory: {os.getcwd()}")
print(f"Parent directory exists: {os.path.exists('..')}")
print(f"Utils directory exists: {os.path.exists('../utils')}")

from utils.prompts import render
from utils.llm_client import LLMClient #pick llm client
from utils.logging_utils import log_llm_call #log every api call
from utils.router import  pick_model, should_use_reasoning_model #general, reasoning, strrong



Current directory: c:\Users\viraj\Zuu\AI_Engineer_Bootcamp\Week_01\notebooks
Parent directory exists: True
Utils directory exists: True


In [ ]:
os.getcwd()

### 01- Zero Shot

In [ ]:
prompt_text , spec = render(
    "zero_shot.v1",
    role = "sentiment_analyst",
    instruction = "Analyze the following text and determine the sentiment as positive/negative/neutral",
    constraints = "the sentiment should be one of the following: positive, negative, neutral",
    format = "the sentiment is : {sentiment}"
    )


model = pick_model("groq", "general")
llm = LLMClient("groq", model)

text = "It seems nice but i dont like it "
messages=[
        {"role": "system", "content": "You must respond in EXACTLY this format with no additional text: 'the sentiment is : [positive/negative/neutral]'"},
        {"role": "user", 
        "content": f"{prompt_text} Review: {text}"
        }
    ]

llm.chat(messages, temperature=0.0)

### 02_Few Shot

provide an exmple

In [ ]:
examples = """
example 01:
review: "I absolutely love this product! It has changed my life for the better."
sentiment: positive

example 02:
review: "This is the worst purchase I've ever made. Completely disappointed."
sentiment: negative

example 03:
review: "The product is okay, nothing special but not terrible either."
sentiment: neutral

example 04:
review: "im really happy with the product its bad"  
sentiment: negative

example 05:
review: "The service was excellent and the staff were very friendly."
sentiment: positive

example 06:
review: "im really unhappy with the product but its amazing"  
sentiment: positive
"""

prompt_text , spec = render(
    "few_shot.v1",
    role = "sentiment_analyst",
    examples = examples,
    query = "",
    constraints = "the sentiment should be one of the following: positive, negative, neutral",
    format = "the sentiment is : {sentiment}"
    )


model = pick_model("groq", "general")
llm = LLMClient("groq", model)

text = "It seems nice but i dont like it"
messages=[
        {"role": "system", "content": "You must respond in EXACTLY this format with no additional text: 'the sentiment is : [positive/negative/neutral]'"},
        {"role": "user", 
        "content": f"{prompt_text} Review: {text}"
        }
    ]

llm.chat(messages, temperature=0.0)

### 03_COT

In [ ]:
# CoT auto-routes to reasoning model
reasoning_model = pick_model('groq', 'cot')
print(f'Using reasoning model: {reasoning_model}')

client_reasoning = LLMClient('groq', reasoning_model)

# Problem from live class: break time vs travel time confusion
problem = """A car travels 100 miles in 2 hours. 

Question 1: What is the average speed of the car?
Question 2: If the car stopped for 40 minutes during this 2-hour journey, 
what was the average speed during the actual driving time?

Important: The 2 hours already includes the 40-minute stop."""

# Additional guidance
instruction = """Solve the following problem step by step.
1. First identify whether the car travelled the entire time without stopping or not.
2. If car stopped for x minutes and overall travelled for y hours, the actual driving duration is y-x.
3. If stopping time x is mentioned, do not add it to the travel duration because it's already included.
   So actual travel time is y (total time) - x (stopping time).
4. If car travelled the entire time without stopping, then average speed is distance / y.
5. If car stopped for x minutes, then average speed during driving is distance / (y - x)."""

prompt_text, spec = render(
    'cot_reasoning.v1',
    role='math tutor',
    problem=problem
)

# Combine problem with instruction 
full_prompt = f"""text: {prompt_text}

instruction: {instruction}"""

messages = [{'role': 'user', 'content': full_prompt}]
response = client_reasoning.chat(messages, temperature=spec.temperature, max_tokens=spec.max_tokens)

print('CoT Response (Travel Time with Break):')
print('=' * 80)
print(response['text'])
print('=' * 80)
print(f"\nUsage: {response['usage']}")
print(f"Latency: {response['latency_ms']}ms")
log_llm_call('groq', reasoning_model, 'cot', response['latency_ms'], response['usage'])

### Structured outputs and JSON schema

#### part1-Json

In [2]:
from utils.json_utils import safe_parse_json, validate_json_schema, create_simple_schema, format_schema_for_prompt

In [4]:
text = """The CloudSync Pro Business plan costs 20 LKR per user per month.
It includes 10TB storage and is currently available."""

schema = create_simple_schema({
                            "name": "string",
                            "price": "number",
                            "currency": "string",
                            "in_stock": "boolean"
                        }, required=["name", "price", "currency"])
print(schema)




{'type': 'object', 'properties': {'name': {'type': 'string'}, 'price': {'type': 'number'}, 'currency': {'type': 'string'}, 'in_stock': {'type': 'boolean'}}, 'required': ['name', 'price', 'currency']}


In [7]:
prompt_text, spec = render("json_extract.v1", schema=schema, text=text)
print(prompt_text)
model = pick_model('groq', 'general')
client = LLMClient('groq', model)

Extract the requested fields and return ONLY valid JSON matching this schema:
{'type': 'object', 'properties': {'name': {'type': 'string'}, 'price': {'type': 'number'}, 'currency': {'type': 'string'}, 'in_stock': {'type': 'boolean'}}, 'required': ['name', 'price', 'currency']}

Text:
The CloudSync Pro Business plan costs 20 LKR per user per month.
It includes 10TB storage and is currently available.

Return ONLY JSON, no extra text.


In [8]:
response = client.json_chat(
    [
        {"role": "user", "content": prompt_text}
    ],
    temperature=0.0
)['text']

In [11]:
from utils.json_utils import safe_parse_json


success, data, error = safe_parse_json(response)
print(data)

{'name': 'CloudSync Pro Business', 'price': 20, 'currency': 'LKR', 'in_stock': True}


#### part2- Pydantic Models

In [12]:
from pydantic import BaseModel, Field
from utils.json_utils import (
                            format_pydantic_schema_for_prompt,
                            parse_json_with_pydantic
)

class ProductInfo(BaseModel):
    name: str = Field(..., description="The name of the product")
    price: float = Field(..., description="The price of the product")
    currency: str = Field(..., description="The currency of the product")
    in_stock: bool = Field(..., description="Whether the product is in stock")


schema_str = format_pydantic_schema_for_prompt(ProductInfo)
print(schema_str)

{
  "properties": {
    "name": {
      "description": "The name of the product",
      "title": "Name",
      "type": "string"
    },
    "price": {
      "description": "The price of the product",
      "title": "Price",
      "type": "number"
    },
    "currency": {
      "description": "The currency of the product",
      "title": "Currency",
      "type": "string"
    },
    "in_stock": {
      "description": "Whether the product is in stock",
      "title": "In Stock",
      "type": "boolean"
    }
  },
  "required": [
    "name",
    "price",
    "currency",
    "in_stock"
  ],
  "title": "ProductInfo",
  "type": "object"
}


In [14]:
from pandas.io.formats.printing import _pprint_seq


text = """The CloudSync Pro Business plan costs $20 per user per month.
It includes 10TB storage and is currently available."""

prompt_text, spec = render("json_extract.v1", schema=schema_str, text=text)

model = pick_model('groq', 'general')
client = LLMClient('groq', model)

response = client.json_chat(
    [
        {"role": "user", "content": prompt_text}
    ],
    temperature=0.0
)['text']
print(response)


{
  "name": "CloudSync Pro Business",
  "price": 20,
  "currency": "USD",
  "in_stock": true
}


In [ ]:
success, data, error = safe_parse_json(response)
print(data)